- General semantics    
Two tensors are “broadcastable” if the following rules hold:
- Each tensor has at least one dimension.

- When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

In [1]:
import torch
x = torch.empty(5,7,3)
y = torch.empty(5,7,3)
# same shapes are always broadcastable (i.e. the above rules always hold)

In [ ]:
x = torch.empty((0,))
y = torch.empty(2,2)
# x and y are not broadcastable, because x does not have at least 1 dimension

In [ ]:
# can line up trailing dimensions
x = torch.empty(5,3,4,1)
y = torch.empty(3,1,1)
# x and y are broadcastable.
# 1st trailing dimension: both have size 1
# 2nd trailing dimension: y has size 1
# 3rd trailing dimension: x size == y size
# 4th trailing dimension: y dimension doesn't exist

In [ ]:
# but:
x = torch.empty(5,2,4,1)
y = torch.empty(3,1,1)
# x and y are not broadcastable, because in the 3rd trailing dimension 2 != 3

If two tensors x, y are “broadcastable”, the resulting tensor size is calculated as follows:

If the number of dimensions of x and y are not equal, prepend 1 to the dimensions of the tensor with fewer dimensions to make them equal length.

Then, for each dimension size, the resulting dimension size is the max of the sizes of x and y along that dimension.

In [2]:
# can line up trailing dimensions to make reading easier
x = torch.empty(5,1,4,1)
y = torch.empty(  3,1,1)
(x + y).size()

torch.Size([5, 3, 4, 1])

In [3]:
# but not necessary:
x = torch.empty(1)
y = torch.empty(3,1,7)
(x + y).size()

torch.Size([3, 1, 7])

In [4]:
x = torch.empty(5,2,4,1)
y = torch.empty(3,1,1)
(x + y).size()

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

- In-place semantics    
  One complication is that in-place operations do not allow the in-place tensor to change shape as a result of the broadcast.    
  一个复杂的问题是，就地操作不允许就地张量在广播的结果下改变形状

In [9]:
x = torch.empty(5,3,4,1)
y = torch.empty(3,1,1)
(x.add_(y)).size()

torch.Size([5, 3, 4, 1])

In [8]:
# but:
x = torch.empty(1,3,1)
y = torch.empty(3,1,7)
(x.add_(y)).size()

RuntimeError: output with shape [1, 3, 1] doesn't match the broadcast shape [3, 3, 7]

- Backwards compatibility    
  PyTorch 的早期版本允许某些逐点函数在形状不同的张量上执行，只要每个张量中的元素数量相等。然后，逐点操作将通过将每个张量视为一维来执行。现在，PyTorch 支持广播，而“一维”逐点行为已被弃用，当张量不可广播但具有相同数量的元素时，将生成 Python 警告

In [ ]:
# 请注意，引入广播可能会导致在不具有相同形状但可广播且具有相同元素数量的两个张量之间出现向后不兼容的更改。例如：
torch.add(torch.ones(4,1), torch.randn(4))
# 之前会生成大小为：torch.Size([4,1]) 的 Tensor，但现在生成大小为：torch.Size([4,4]) 的 Tensor。


tensor([[1.2891, 1.4053, 3.2596, 0.4619],
        [1.2891, 1.4053, 3.2596, 0.4619],
        [1.2891, 1.4053, 3.2596, 0.4619],
        [1.2891, 1.4053, 3.2596, 0.4619]])

In [15]:
# 为了帮助您识别代码中可能存在的由广播引起的向后不兼容性案例，
# 您可以设置 torch.utils.backcompat.broadcast_warning.enabled 为 True，这将在这种情况下生成 Python 警告。
torch.utils.backcompat.broadcast_warning.enabled = True
torch.add(torch.ones(4,1),torch.ones(4))

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])